# Day 5-Part 1: Well logs

Although quite local, well data, logs and cores, are the closest observations we have of the subsurface (with the exception of caves at shallow depths). In this notebook, we will analyse a set of well logs from the Central Graben, North Sea. The dataset consists of 39 well logs, and one Excel spreadsheet containing the location of the wells and the top and base of the Rogaland Group (Paleocene-Eocene).

The well logs are from NPD's [Diskos](https://www.npd.no/en/diskos/) public portal. This dataset is analyzed in [Adrian Kirkhus' UiS Master thesis](https://uis.brage.unit.no/uis-xmlui/handle/11250/3024570).

Let's start with the preliminars:

In [ ]:
# 1. Import libraries
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

2. Download the data from the following **link**. Unzip the file and put the `rogaland_group` folder in the `day5/data` folder.

**Note**: *Notice that since the data are from NPD-Diskos, I can't provide it in the repository. If you are in Norway, please contact [me](mailto:nestor.cardozo@uis.no) and I can provide the link.*

Now, let's import the spreadsheet containing the location of the wells, and top and base of the Rogaland group:

In [ ]:
# import Rogaland Gp coordinates as pandas DataFrame
rg_gp = pd.read_excel(os.path.join("..", "data", "rogaland_group", "RogalandGp.xlsx"))
rg_gp.head()

Let's plot the wells' locations:

In [ ]:
# plot wells" locations

# make figure
fig, ax = plt.subplots(figsize=(8,6))

# plot wells and labels
for i in range(rg_gp.shape[0]):
    ax.plot(rg_gp.iloc[i,1], rg_gp.iloc[i,2], "ro", markersize=4)
    ax.text(rg_gp.iloc[i,1]-2000, rg_gp.iloc[i,2]+750, rg_gp.iloc[i,0])

ax.axis("equal") # make axes equal

# set axes labels
ax.set_xlabel("East[m]")
ax.set_ylabel("North[m]");

Well log information is commonly stored in the Log ASCII Standard (LAS) format. A single las file can only contain data for one well, but it can contain any number of logs (called 'curves') from that well. Let's get a list of the las files in the dataset. Here we use the `glob.glob` method to obtain a list of las files in the dataset directory:

In [ ]:
# define paths to files and print number of files
las_files = glob.glob(os.path.join("..", "data", "rogaland_group", "*.las"))
print("found", len(las_files), "las files")

So there are 39 wells. In Python, we can use the library [lasio](https://github.com/kinverarity1/lasio) to read and write las files. This library does not come with Anaconda, so we will need to install it:

In [ ]:
# run this cell if lasio is not installed
import sys
!{sys.executable} -m pip install lasio

Now we can import and use `lasio` to read the wells. The single line of code below will do the job. Be patient, it will take some time to read all the files:

In [ ]:
import lasio

# read the wells
las_objs = [lasio.read(f) for f in las_files]

Let's print the wells names and curves (logs) in each well:

In [ ]:
# print well names and logs mnemonics
for w in las_objs:
    print(w.header["Well"]["WELL"].value, "=", w.keys())

Let's plot the high resolution gamma ray log (HGR, API units). In the code we basically:

1. Choose one well from the list of `las_objs`
2. Find out the top and base of the rogaland group at the well from the `rg_gp` DataFrame
3. Extract the well as a DataFrame and restrict it to the Rogaland Group
4. Plot the HGR. We also fill the left part of the curve with green (shale), and the right one with yellow (sandstone)

In [ ]:
# plot HGR of one well

las_id = 0 # element index in las_objs

# well
w = las_objs[las_id]
w_name = w.header["Well"]["WELL"].value

# index at which well column in rg_gp dataframe = w_name
rg_gp_id = rg_gp[rg_gp["well"] == w_name].index.values[0]

# depth of top and base of the Rogaland group
top_rg = rg_gp.loc[rg_gp_id, "TVD top"]
base_rg = rg_gp.loc[rg_gp_id, "TVD base"]

# extract well as DataFrame
df = w.df()
df["DEPTH"] = df.index

# restrict to Rogaland group
df_rg = df[(df["DEPTH"] >= top_rg) & (df["DEPTH"] <= base_rg)]

# plot HGR
if "HGR" in df_rg.columns:
    fig, ax = plt.subplots(figsize=(3,10))
    df_rg.plot(x="HGR", y="DEPTH", c="black", lw=0.5, legend=False, ax=ax)
    ax.fill_betweenx(df_rg["DEPTH"], df_rg["HGR"], 0, facecolor="green")
    ax.fill_betweenx(df_rg["DEPTH"], df_rg["HGR"], 150, facecolor="yellow")
    
    # set axis
    ax.set_ylim(base_rg,top_rg) # y increases downwards
    ax.set_xlim(0,150)
    ax.set_title("HGR of well " + w_name)
    ax.grid(which="both");

Since the well is a DataFrame, we can quickly get a statistics for all the logs:

In [ ]:
df_rg.describe()

Now, let's plot all the wells. This code is similar to the one above, but in this case we iterate over all wells in `las_objs`. Since we are plotting many wells, we don't fill in areas with colors:

In [ ]:
# plot HGR of all wells in Rogaland group interval
fig, ax = plt.subplots(figsize=(8,10))
for w in las_objs:
    w_name = w.header["Well"]["WELL"].value
    
    # index at which well column in rg_gp dataframe = w.name
    rg_gp_id = rg_gp[rg_gp["well"] == w_name].index.values[0]
    
    # depth of top and base of the Rogaland group
    top_rg = rg_gp.loc[rg_gp_id, "TVD top"]
    base_rg = rg_gp.loc[rg_gp_id, "TVD base"]
    
    # extract well as DataFrame
    df = w.df()
    df["DEPTH"] = df.index
    
    # restrict to Rogaland group
    df_rg = df[(df["DEPTH"] >= top_rg) & (df["DEPTH"] <= base_rg)]
    
    # plot HGR of well
    if "HGR" in df_rg.columns:
        df_rg.plot(x="HGR", y="DEPTH", lw=0.5, legend=False, ax=ax)
    
# set axis
ax.invert_yaxis()
ax.set_ylabel("Depth [m]")
ax.set_xlim(0,150)
ax.set_title("HGR of all wells")
ax.grid(which="both");

This displays nicely the variation of gamma ray with depth in all wells. Let's print the HGR statistics of all wells:

In [ ]:
# HGR statistics of all wells
print("HGR statistics of all wells")

for w in las_objs:
    w_name = w.header["Well"]["WELL"].value
    
    # index at which well column in rg_gp dataframe = w.name
    rg_gp_id = rg_gp[rg_gp["well"] == w_name].index.values[0]
    
    # depth of top and base of Rogaland Gp
    top_rg = rg_gp.loc[rg_gp_id, "TVD top"]
    base_rg = rg_gp.loc[rg_gp_id, "TVD base"]
    
    # extract well as DataFrame
    df = w.df()
    df["DEPTH"] = df.index
    
    # restrict to Rogaland Gp
    df_rg = df[(df["DEPTH"] >= top_rg) & (df["DEPTH"] <= base_rg)]
    
    # print HGR statistics
    if "HGR" in df_rg.columns:
        print(w_name, "mean = {:.1f}, std = {:.1f}, min = {:.1f}, 25% = {:.1f}, 50% = {:.1f}, 75% = {:.1f}, max = {:.1f}"
              .format(df_rg["HGR"].mean(), df_rg["HGR"].std(), df_rg["HGR"].min(),
                     df_rg["HGR"].quantile(0.25), df_rg["HGR"].quantile(0.50), 
                     df_rg["HGR"].quantile(0.75), df_rg["HGR"].max()))

The net to gross (ntg) is the ratio of the total amount of sandstone divided by the total thickness of the reservoir interval. The simplest way to calculate ntg is using a gamma ray value as a threshold for shale. Gamma ray values larger or equal than this threshold are shale, and those lower than the threshold are sandstone. For the Rogaland Group and the dataset under consideration, Adrian Kirkhus used a threshold of 40 API. Let's do the same. In the code below, we store the calculated ntg values in a column of the `rg_gp` DataFrame:

In [ ]:
gr_val = 40 # gamma ray value that separates sand from shale 

rg_gp["ntg"] = 0 # add column to rg_gp DataFrame

# Compute net to gross of all wells
print("Net to Gross of all wells")

for w in las_objs:
    w_name = w.header["Well"]["WELL"].value
    
    # index at which well column in rg_gp dataframe = w.name
    rg_gp_id = rg_gp[rg_gp["well"] == w_name].index.values[0]
    
    # depth of top and base of Rogaland Gp
    top_rg = rg_gp.loc[rg_gp_id, "TVD top"]
    base_rg = rg_gp.loc[rg_gp_id, "TVD base"]
    
    # extract well as DataFrame
    df = w.df()
    df["DEPTH"] = df.index
    
    # restrict to Rogaland Gp
    df_rg = df[(df["DEPTH"] >= top_rg) & (df["DEPTH"] <= base_rg)]
    
    # estimate net to gross
    if "HGR" in df_rg.columns:
        net = np.sum(df_rg["HGR"] < gr_val) # net = samples where gr < gr_val 
        shale = np.sum(df_rg["HGR"]>= gr_val) # shale = samples where gr >= gr_val
        ntg = net/(net+shale) * 100 # net to gross
        print("Net to gross in well", w_name, "=", "{:.2f}".format(ntg)) # print net to gross
        rg_gp.loc[rg_gp_id, "ntg"] = ntg # add value to rg_gp DataFrame

The `rg_gp` DataFrame has now a column labelled `ntg`:

In [ ]:
rg_gp.head()

Let's use the DataFrame to plot the distribution of ntg in the area. We represent graphically ntg using a pie diagram:

In [ ]:
# plot ntg

pi = np.pi # pi

# make figure
fig, ax = plt.subplots(figsize=(12,8))

# for all wells
for i in range(rg_gp.shape[0]):
    # first define the ratio, which in this case is ntg/100
    r1 = rg_gp.iloc[i,5]/100
    
    # calculate the points of the first pie marker
    # these are just the origin (0, 0) + some (cos, sin) points on a circle
    x1 = np.cos(2 * pi * np.linspace(0, r1))
    y1 = np.sin(2 * pi * np.linspace(0, r1))
    xy1 = np.row_stack([[0, 0], np.column_stack([x1, y1])])
    s1 = np.abs(xy1).max()
    
    # calculate the points of the second pie marker
    x2 = np.cos(2 * pi * np.linspace(r1, 1))
    y2 = np.sin(2 * pi * np.linspace(r1, 1))
    xy2 = np.row_stack([[0, 0], np.column_stack([x2, y2])])
    s2 = np.abs(xy2).max()

    # plot pie diagram
    ax.scatter(rg_gp.iloc[i,1], rg_gp.iloc[i,2], marker=xy1, s=s1**2 * 200, facecolor="yellow")
    ax.scatter(rg_gp.iloc[i,1], rg_gp.iloc[i,2], marker=xy2, s=s2**2 * 200, facecolor="green")
    # plot well name
    ax.text(rg_gp.iloc[i,1]-1500, rg_gp.iloc[i,2]+1200, rg_gp.iloc[i,0])
    
# set axes
ax.axis("equal") # make axes equal
ax.set_facecolor("#aed6f1") # change background color
ax.set_xlabel("East[m]") # x label
ax.set_ylabel("North[m]"); # y label

We still need to make 'geological' sense of these data, perhaps by superimposing the figure above to a map with the geological provinces of the area. For comparison, you can check Fig. 15 of [Adrian Kirkhus thesis](https://uis.brage.unit.no/uis-xmlui/handle/11250/3024570).